# HW6 - Character Classification
Anirudh Lath | CS6017 | July 24, 2022
In this assignment we'll tackle a slightly more complicated image classification problem than MNIST digit classification. We're going to classify characters that contain (gasp!) letters!

The dataset we'll play with is from University of California, Irvine (UCI) and contains a bunch of images of letters of various fonts. Some printed + scanned, some the values screen-capped from a computer. The images are 20x20 pixels, grayscale.

In [1]:
# Prepare the notebook
import torchvision
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim

from sklearn.model_selection import train_test_split

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))]) #convert from images to tensors
mnist_test  = torchvision.datasets.MNIST( "./mnist", train=False, download=True, transform=transform )

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3080
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


## Step 1: Data Acquisition + Cleanup
Import the data for Arial font

In [2]:
df = pd.read_csv('fonts/ARIAL.csv')
df.head()

,font,fontVariant,m_label,strength,italic,orientation,m_top,m_left,originalH,originalW,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,ARIAL,scanned,48,0.4,0,0.0,0,0,15,25,...,255,255,255,154,1,1,1,1,1,1
1,ARIAL,scanned,50,0.4,0,0.0,0,0,18,11,...,213,225,229,239,240,240,221,209,116,95
2,ARIAL,scanned,83,0.4,0,0.0,0,0,13,10,...,255,255,255,255,255,255,1,1,1,1
3,ARIAL,scanned,48,0.4,0,0.0,0,0,15,18,...,1,1,1,1,1,1,1,1,1,1
4,ARIAL,scanned,54,0.4,0,0.0,0,0,14,30,...,255,255,255,255,1,1,1,1,1,1


Drop all columns except m_label and the pixel values which are scattered across 400 columns labeled rxcy (where x and y are the row and column numbers that range from 0 to 19).

In [3]:
df.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation', 'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'], inplace=True)
df

,m_label,r0c0,r0c1,r0c2,r0c3,r0c4,r0c5,r0c6,r0c7,r0c8,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,48,1,1,1,1,1,1,154,255,255,...,255,255,255,154,1,1,1,1,1,1
1,50,4,7,22,49,97,120,139,156,162,...,213,225,229,239,240,240,221,209,116,95
2,83,1,1,1,1,255,255,255,255,255,...,255,255,255,255,255,255,1,1,1,1
3,48,1,1,1,1,1,114,255,255,255,...,1,1,1,1,1,1,1,1,1,1
4,54,1,1,1,1,1,1,1,1,255,...,255,255,255,255,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26232,37,1,4,83,208,255,255,161,16,1,...,1,1,16,161,255,255,208,83,4,1
26233,36,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
26234,35,1,1,1,1,1,1,77,253,255,...,255,255,253,77,1,1,1,1,1,1
26235,34,43,255,255,255,255,255,255,255,255,...,43,255,255,255,255,213,1,1,1,1


Now, write a function that takes in one of these types of dataframe and returns 2 numpy arrays: Xs which is a #samples x 20 x 20 array containing the pixel values, and Ys which is a #samples x 1 array containing the ascii vales for each character. You should normalize the Xs array so the values go from 0-1 (most likely this requires dividing by 255).

In [4]:
def extract_data(df):
    #Xs which is a #samples x 20 x 20 array containing the pixel values
    X = df.drop(columns='m_label').to_numpy(dtype=np.float64)
    X = np.array([x.reshape(20, 20) for x in X], dtype=np.float64) / 255
    X = np.reshape(X, (-1, 1, 20, 20))

    #Ys
    Y_data = df["m_label"].to_numpy()
    keys, Y = np.unique(Y_data, return_inverse=True)
    # Y = np.array(Y_data)

    return X, Y , keys

X, Y, keys = extract_data(df)

## Step 2: Build a PyTorch Network
We're going to use the PyTorch library, like we've seen in class, to build/train our network. Check out the notebooks we've made in class or the official documentation/tutorials.

To start with, we're going to use a model very similar to the MNIST CNN we used in class. It will consist of:

* a Convolution2D layer with ReLU activations
* a max pooling layer
* another convolution layer
* another max pooling layer
* a dense layer with relu activation
* a dense layer

Compile and train your network like we did in class. You'll probably have to use the np.reshape() function on your data to make PyTorch happy. I reshaped my X values like np.reshape(Xs, (-1, 1, 20, 20)) to get them in the right format.

For training, you'll want to check out torch.utils.data.DataLoader which can take a TensorDataset so you can iterate over batches like we did in class for the MNIST data.

### Create the network

In [5]:
class network1(nn.Module):
    def __init__(self):
        super(network1, self).__init__()

        self.convolution1 = nn.Conv2d(1, 8, 3)
        self.pooling1 = nn.MaxPool2d(2, 2)
        self.dense1 = nn.Linear(576, 4000)

        self.convolution2 = nn.Conv2d(8, 64, 3)
        self.pooling2 = nn.MaxPool2d(2, 2)
        self.dense2 = nn.Linear(4000, 3098)


    def forward(self, x):
        x = self.pooling1(F.relu(self.convolution1(x)))
        x = self.pooling2(F.relu(self.convolution2(x)))

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        return x

    def num_flat_features(self, x):
        # Exclude Batch Dimension
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


network1 = network1().to(device)

### Train the model

In [6]:
def train( model, epochs, data, labels ):
    print("Training the model, please wait...")

    criterion = nn.CrossEntropyLoss()

    # use the optimiser to find weights
    optimizer = optim.Adam( model.parameters(), lr= 1e-4 )

    model.float()

    for epoch in range( epochs ):

        running_loss = 0.0
        optimizer.zero_grad()

        outputs = model(data.float()) # Predict outputs
        loss = criterion(outputs, labels) # Check the predictions accuracy

        loss.backward() # Calculate new weights
        optimizer.step() # Change weights and try again

        running_loss += loss.item()

    print("Training complete.")

def evaluate( model, data, labels ):
    #load some test data
    correct = 0
    total = 0

    with torch.no_grad(): # Don't calculate gradients as it's not necessary here.

        outputs = model(data.float())

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print( 'Accuracy of the network on Arial Font: %d%%' % (100 * correct / total))

In [7]:
data = torch.from_numpy(X)
labels = torch.from_numpy(Y)
data, labels = data.to(device), labels.to(device)

print(min(labels))
print(max(labels))


train(network1, 1500, data, labels) # Training network with an epoch value of 1500, takes a long time, so set it to 1 for testing.

tensor(0, device='cuda:0')
tensor(3097, device='cuda:0')


### Evaluate the model

In [24]:
print("Evaluating the model, please wait...")
evaluate(network1, data, labels)
print("Evaluation complete.")

Evaluating the model, please wait...
Accuracy of the network on Arial Font: 91 %
Evaluation complete.


## Step 3: Exploration and Evaluation
### Cross-validation (Training/Test Splits)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.5, shuffle=True)

train(network1, 500, X_train, y_train)

evaluate(network1, X_test, y_test)


Training the model, please wait...
Training complete.
Accuracy of the network on Arial Font: 82%


Seems like if we train the network for 500 epochs that accuracy with cross-validation is about 83%, but in the next cell, I will increase epoch value to 1000 and then see what that does, probably increased accuracy.

In [38]:
train(network1, 1000, X_train, y_train)

evaluate(network1, X_test, y_test)

Training the model, please wait...
Training complete.
Accuracy of the network on Arial Font: 80%


Turns out the accuracy went down by 2%. What if I evaluate against train data?

In [39]:
evaluate(network1, X_train, y_train)

Accuracy of the network on Arial Font: 94%


### Trying a different network topology

In [8]:
class network2(nn.Module):
    def __init__(self):
        super(network2, self).__init__()

        self.convolution1 = nn.Conv2d(1, 8, 2)
        self.pooling1 = nn.MaxPool2d(2, 2)
        self.dense1 = nn.Linear(128, 5000)

        self.convolution2 = nn.Conv2d(8, 64, 2)
        self.pooling2 = nn.MaxPool2d(2, 2)
        self.dense2 = nn.Linear(5000, 3098)

        self.convolution3 = nn.Conv2d(64, 128, 2)
        self.pooling3 = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout()


    def forward(self, x):
        x = self.pooling1(F.relu(self.convolution1(x)))
        x = self.dropout(x)
        x = self.pooling2(F.relu(self.convolution2(x)))
        x = self.dropout(x)
        x = self.pooling3(F.relu(self.convolution3(x)))

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        return x

    def num_flat_features(self, x):
        # Exclude Batch Dimension
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


network2 = network2().to(device)

data = torch.from_numpy(X)
labels = torch.from_numpy(Y)
data, labels = data.to(device), labels.to(device)
torch.cuda.empty_cache()
train(network2, 1500, data, labels)
evaluate(network2, data, labels)

Training the model, please wait...


KeyboardInterrupt: 

In [15]:
torch.cuda.empty_cache()